# Exercise: Simplex pivots in the short tableau
<font color='blue'><b>Goal:</b></font>
Writing a function to perform one pivot step in the short tableau.

<font color='blue'><b>Required packages:</b></font>`numpy`

Suppose you are given the following LP from the lecture notes:

$$\begin{equation}\label{eq:sampleTermLp}
\begin{array}{rrcrcll}
\max z = &400 x_1 &+ &900 x_2 &       &    & \\
         &x_1     &+ &4 x_2   & \leq  &40  & \text{(constraint 1)} \\
         &2x_1    &+ & x_2    & \leq  &42  & \text{(constraint 2)}\\
         &1.5x_1  &+ &3x_2    & \leq  &36  & \text{(constraint 3)}\\
         &x_1     &  &        & \geq  &0   & \text{(non-negativity for $x_1$)}\\
         &        &  &x_2     & \geq  &0   & \text{(non-negativity for $x_2$)}\\
\end{array}
\end{equation}$$
This problem's graphical presentation looks as follows:

![](pivoting_fig1.png)

After adding slack variables, you obtain the short tableau
$$\begin{array}{c|rr|r}
&x_1&x_2&1\\
\hline
z&-400&-900&0\\
\hline
y_1&1&4&40\\
y_2&2&1&42\\
y_3&1.5&3&36
\end{array}$$

We will implement a function to perform pivoting steps on short tableaus as for example the one above.

## Additional `numpy` functionality
For your task, you will need to perform row operations. When working with matrices in `numpy`, this can be done as follows. Let us first create a matrix $A$:

In [1]:
import numpy as np
A = np.matrix(
[
    [1,2],
    [3,4],
    [5,6]
])
A

matrix([[1, 2],
        [3, 4],
        [5, 6]])

To obtain the first row, we type

In [2]:
A[0,:]

matrix([[1, 2]])

Subtracting the first row from the second, for example, can be done with the code

In [3]:
A[1,:] = A[1,:] - A[0,:]
A

matrix([[1, 2],
        [2, 2],
        [5, 6]])

To get the number of rows of $A$, type

In [4]:
A.shape[0]

3

Analogously, we can select columns as follows:

In [5]:
A[:,0]

matrix([[1],
        [2],
        [5]])

To get the number of columns of $A$, type

In [6]:
A.shape[1]

2

Finally, to obtain an entry $A_{i,j}$, type (exemplified for $i=0$, $j=1$):

In [7]:
a_01 = A[0,1]
a_01

2

Furthermore, you may need to append a standard unit vector $v$ to a matrix $A$. This can be done as follows, where we exemplify the procedure with the first standard unit vector $e_1$:

In [8]:
v = np.transpose(np.matrix(np.zeros(A.shape[0])))
v[0,] = 1 # first standard unit vector

A = np.concatenate((v,A),axis=1)
A

matrix([[1., 1., 2.],
        [0., 2., 2.],
        [0., 5., 6.]])

Finally, to delete a column $i$, let's say for $i=0$ (the first column), type:

In [9]:
A = np.delete(A,[0],1)
A

matrix([[1., 2.],
        [2., 2.],
        [5., 6.]])

## A pivoting function
We now want to implement a function `pivot(T, i, j)` which takes in a short tableau $T$, and replaces the basis variable corresponding to row $i$ of the tableau by the variable corresponding to column $j$ of the tableau. By convention, the tableau $T$ should include the objective function row as the first row (the row with index $0$), and the right hand sides as the last column.

To avoid accidental selection of a pivot from the objective function row or the column of right-hand sides, your function should check that the input indices $i$ and $j$ are indeed pointing at a potential pivot. If not, the function should print a message saying so, and return the input tableau. Similarly, if the indicated pivot element is zero (in which case the pivoting step cannot be performed), the function should print a message saying so, and return the input tableau.
If a non-zero pivot is given, the pivoting step should be carried out, and the new tableau should be returned. Additionally, the function should print a message indicating whether or not the pivot was legal for phase II of the simplex method.

Complete the given framework below to obtain the pivoting function with the desired functionality.

In [21]:
def pivot(T,i,j):
    
    # dimensions of the coefficient matrix
    m = T.shape[0] - 1 # number of row
    n = T.shape[1] - 1 # number of column
    print(f"m={m}, n={n}")
    
    if not (1 <= i <= m and 0 <= j <= n-1): # Check if the indices are not pointing at a potential pivot
        print("[Error] Given indices are not pointing at a potential pivot.")
        return T

    if (T[i,j] == 0): # Check if the indicated pivot is zero
        print("[Error] Given pivot is zero, step cannot be performed.")
        return T
    
    # Check if indicated pivot is legal for phase II of Simplex Method
    if (T[0,j]<0 and 
        T[i,n]/T[i,j]==min([T[l,n]/T[l,j] for l in range(1,m+1) if T[l,j]>0])):
        print("[Info] Given pivot is legal.")
    else:
        print("[Warning] Given pivot is illegal.")
    
    # Implement pivoting step here
    # 1. Append a unit vector to the tableau
    e = np.matrix(np.zeros([1+m,1]))
    e[i,] = 1
    # To put this extra vector on the right side is more safe. (Avoid index confusion)
    T_pivoted = np.concatenate((T,e),axis=1) 
    # 2. Do Row Operation, Notice that after this operation, the jth column becomes unit column, 
    #    and the appended column becomes the now j-th column
    for k in range(0,T_pivoted.shape[0]):
        if k == i: # pivot row
            T_pivoted[k,:] = T_pivoted[k,:]/T_pivoted[k,j]
        else: # Non pivor row
            T_pivoted[k,:] = T_pivoted[k,:] - T_pivoted[k,j]*T_pivoted[i,:]/T_pivoted[i,j]
    # 3. Move the first column to the pivoted j-th column and delete the original one (Now becomes unit vectors already)
    T_pivoted[:,j] = T_pivoted[:,T_pivoted.shape[1]-1]
    T_pivoted = np.delete(T_pivoted,[T_pivoted.shape[1]-1],1) # 1 represents the axis: column
    

    return T_pivoted

In [22]:
np.zeros([4,1])

array([[0.],
       [0.],
       [0.],
       [0.]])

## Testing your implementation
To check whether your implementation works, use your function to reproduce the following pivot steps from the lecture notes:

![](pivoting_fig2.png)

This can be done, for example, by running the code below:

In [23]:
import numpy as np
T = np.matrix([
    [-400,-900,0],
    [1,4,40],
    [2,1,42],
    [1.5,3,36]
])

T = pivot(T,2,0)
print("Vertex b")
print(T)

T = pivot(T,3,1)
print("Vertex c")
print(T)

T = pivot(T,1,0)
print("Vertex d")
print(T)

m=3, n=2
[Info] Given pivot is legal.
Vertex b
[[ 2.00e+02 -7.00e+02  8.40e+03]
 [-5.00e-01  3.50e+00  1.90e+01]
 [ 5.00e-01  5.00e-01  2.10e+01]
 [-7.50e-01  2.25e+00  4.50e+00]]
m=3, n=2
[Info] Given pivot is legal.
Vertex c
[[-3.33333333e+01  3.11111111e+02  9.80000000e+03]
 [ 6.66666667e-01 -1.55555556e+00  1.20000000e+01]
 [ 6.66666667e-01 -2.22222222e-01  2.00000000e+01]
 [-3.33333333e-01  4.44444444e-01  2.00000000e+00]]
m=3, n=2
[Info] Given pivot is legal.
Vertex d
[[ 5.00000000e+01  2.33333333e+02  1.04000000e+04]
 [ 1.50000000e+00 -2.33333333e+00  1.80000000e+01]
 [-1.00000000e+00  1.33333333e+00  8.00000000e+00]
 [ 5.00000000e-01 -3.33333333e-01  8.00000000e+00]]


Next, let us check whether your function gives a warning if the indicated pivot is not legal. To this end let us take another look at the graphical represenation of the original LP:

![](pivoting_fig1.png)

We see that the first and the second constraint are never both tight for any point in the polytope. In other words, there is no solution with $y_1=y_2=0$.

Let us check whether your function detects that the pivoting step taking us to the solution with $y_1=y_2=0$ is illegal! To this end, note that we get to this solution from the tableau for "verted $d$" above if we pivot such that $y_2$ exits the basis and $y_3$ enters the basis. The following call executes precisely the pivoting operation corresponding to this basis change (assuming that you didn't change $T$ after the above pivoting steps).

In [24]:
T = pivot(T,1,1)
print(T)

m=3, n=2
[Warning] Given pivot is illegal.
[[ 2.00000000e+02  1.00000000e+02  1.22000000e+04]
 [-6.42857143e-01 -4.28571429e-01 -7.71428571e+00]
 [-1.42857143e-01  5.71428571e-01  1.82857143e+01]
 [ 2.85714286e-01 -1.42857143e-01  5.42857143e+00]]


Now, let us see whether your function correctly gives an error if a pivot operation cannot be performed. To this end, consider the following short tableau (which is unrelated to the LP above) taken from the script, and try to replace $y_1$ by $y_2$.

$$\begin{array}{c|rr|c}
 &y_3&y_2&1\\
\hline
z&1&1&3\\
\hline
y_1&1&0&3\\
x_2&2&-1&3\\
x_1&1&-1&1
\end{array}$$

In [25]:
# Apply your function to the tableau above.
T = np.matrix([
    [1,1,3],
    [1,0,3],
    [2,-1,3],
    [1,-1,1]
])
T = pivot(T,1,1)

m=3, n=2
[Error] Given pivot is zero, step cannot be performed.


Finally, you can apply your pivoting functions to other examples from class. By doing tests as the above ones, try to see whether the different cases are recognised correctly by your function!